Data Collection and Preparation

JPM past 2 years (8 quarters)

In [1]:
!pip install gdown


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import gdown

# Define the file IDs for each statement
file_jpm_q3_2024 = '1ixD-cP-MrYB0mFkxM0qWe07FUq4gfDXj'
file_jpm_q2_2024 = '1K8zcF98b18b2suXE8GDS67g6GX5BlSi4'
file_jpm_q1_2024 = '1HmXFdfvXPlzjXUZ6B2GA5_38o_6UCrID'
file_jpm_q4_2023 = '1BfMEP-7oyT9vSOBZBFExzrCXTg7CCmoc'
file_jpm_q3_2023 = '1fB0PsTe0Y5iITnPW-75btf6o53OSHztK'
file_jpm_q2_2023 = '1A7HEOOaGyjpRMnog2oAyb98dqI2HG0Ep'
file_jpm_q1_2023 = '1MswM61xSbNvGBx1zASiq6ZyYTZxQ2HnJ'
file_jpm_q4_2022 = '1LE4hs_mTjmarw22ys5gSD7YYDgcnt5bh'

# Build the download URLs
file_ids = {
    'JPM_Q3_2024.pdf': f'https://drive.google.com/uc?id={file_jpm_q3_2024}',
    'JPM_Q2_2024.pdf': f'https://drive.google.com/uc?id={file_jpm_q2_2024}',
    'JPM_Q1_2024.pdf': f'https://drive.google.com/uc?id={file_jpm_q1_2024 }',
    'JPM_Q4_2023.pdf': f'https://drive.google.com/uc?id={file_jpm_q4_2023}',
    'JPM_Q3_2023.pdf': f'https://drive.google.com/uc?id={file_jpm_q3_2023}',
    'JPM_Q2_2023.pdf': f'https://drive.google.com/uc?id={file_jpm_q2_2023}',
    'JPM_Q1_2023.pdf': f'https://drive.google.com/uc?id={file_jpm_q1_2023}',
    'JPM_Q4_2022.pdf': f'https://drive.google.com/uc?id={file_jpm_q4_2022}'
}

# Download each file
for filename, url in file_ids.items():
    gdown.download(url, filename, quiet=False)

print("Download complete.")

Downloading...
From: https://drive.google.com/uc?id=1ixD-cP-MrYB0mFkxM0qWe07FUq4gfDXj
To: c:\Users\alger\Documents\000. Projects\002. CAM DS\EP\002. Code\JPM_Q3_2024.pdf
100%|██████████| 317k/317k [00:00<00:00, 3.66MB/s]
Downloading...
From: https://drive.google.com/uc?id=1K8zcF98b18b2suXE8GDS67g6GX5BlSi4
To: c:\Users\alger\Documents\000. Projects\002. CAM DS\EP\002. Code\JPM_Q2_2024.pdf
100%|██████████| 281k/281k [00:00<00:00, 4.26MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HmXFdfvXPlzjXUZ6B2GA5_38o_6UCrID
To: c:\Users\alger\Documents\000. Projects\002. CAM DS\EP\002. Code\JPM_Q1_2024.pdf
100%|██████████| 312k/312k [00:00<00:00, 1.32MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BfMEP-7oyT9vSOBZBFExzrCXTg7CCmoc
To: c:\Users\alger\Documents\000. Projects\002. CAM DS\EP\002. Code\JPM_Q4_2023.pdf
100%|██████████| 278k/278k [00:00<00:00, 4.75MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fB0PsTe0Y5iITnPW-75btf6o53OSHztK
To: c:\Users\alger\Documents\0

Download complete.


In [3]:
pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Data extraction using pdfplumber

In [4]:
import pdfplumber
import pandas as pd
import re

# List of files (paths) for the quarterly reports
file_paths = [
    "JPM_Q3_2024.pdf",
    "JPM_Q2_2024.pdf",
    "JPM_Q1_2024.pdf",
    "JPM_Q4_2023.pdf",
    "JPM_Q3_2023.pdf",
    "JPM_Q2_2023.pdf",
    "JPM_Q1_2023.pdf",
    "JPM_Q4_2022.pdf",
    "JPM_Q3_2022.pdf",
    "JPM_Q2_2022.pdf",
    "JPM_Q1_2022.pdf",
    "JPM_Q4_2021.pdf",
    "JPM_Q3_2021.pdf",
    "JPM_Q2_2021.pdf",
    "JPM_Q1_2021.pdf",
    "JPM_Q4_2020.pdf",
]

# Keywords for extraction
keywords = ["NII", "Net Interest Income", "net interest income"]

Functions to split each report in the management discussion/Q&A sections and extract sentences including ROTCE from the maangement discussion

In [5]:
# Function to extract specific section text
def extract_section_text(text, start_section, end_section=None):
    start_idx = text.find(start_section)
    if start_idx == -1:
        return ""
    
    if end_section:
        end_idx = text.find(end_section, start_idx)
        return text[start_idx:end_idx] if end_idx != -1 else text[start_idx:]
    else:
        return text[start_idx:]
# Function to separate sections
def separate_sections(filepath):
    with pdfplumber.open(filepath) as pdf:
        extracted_text = ''
        for page in pdf.pages:
            extracted_text += page.extract_text() or ''

    # Separate sections
    management_discussion_text = extract_section_text(extracted_text, 'MANAGEMENT DISCUSSION SECTION', 'QUESTION AND ANSWER SECTION')
    # Q&A text goes until the end of the document
    qna_text = extract_section_text(extracted_text, 'QUESTION AND ANSWER SECTION')
    
    return management_discussion_text, qna_text

# Function to preprocess text by removing line breaks and normalizing whitespace
def preprocess_text(text):
    """
    Replace all types of whitespace characters (including line breaks) with a single space.
    """
    return re.sub(r'\s+', ' ', text)

# Function to extract sentences containing 'ROTCE' or 'Return on Tangible Equity' and clean them
def extract_rotce_sentences(text):
    
    # Preprocess the text to remove line breaks and normalize whitespace
    cleaned_text = preprocess_text(text)
    
    # Create a regex pattern to match sentences containing any of the keywords
    pattern = rf'[^.?!]*\b(?:{"|".join(keywords)})\b[^.?!]*[.?!]'
    
    # Find all matching sentences in the cleaned text
    matches = re.findall(pattern, cleaned_text, re.IGNORECASE)
    
    # Remove duplicates and strip leading/trailing whitespace
    unique_sentences = set(match.strip() for match in matches)
    
    # Prepare the results as a list of dictionaries
    results = []
    for sentence in unique_sentences:
        # Further ensure that any residual multiple spaces are reduced to single spaces
        cleaned_sentence = re.sub(r'\s+', ' ', sentence)
        results.append({
            "Question": None,
            "Answer": cleaned_sentence,
            "Analyst": None,
            "Respondent": None
        })
    
    return results

Now we extract Question and Answer pairs from qna_text

In [6]:
def extract_qna_pairs_with_names(qna_text):
    """
    Extracts Question and Answer pairs with Analyst and Respondent names from the provided Q&A text.

    Parameters:
        qna_text (str): The raw Q&A text extracted from the report.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted Questions, Answers, Analyst, and Respondent.
    """
    # Step 1: Split the text into blocks using the delimiter (lines with multiple dots)
    blocks = re.split(r'\.{3,}', qna_text)
    
    qna_list = []  # List to store dictionaries of Q&A pairs
    
    # Initialize variables to keep track of the current Q&A context
    current_question = None
    current_analyst = None
    current_answers = []
    current_respondents = set()
    
    for block in blocks:
        block = block.strip()  # Remove leading and trailing whitespace
        if not block:
            continue  # Skip empty blocks

        lines = block.split('\n')  # Split the block into lines
        if len(lines) < 3:
            # Handle cases with insufficient lines, such as Operator blocks
            header = lines[0].strip()
            if header.endswith(' Q') or header.endswith(' A'):
                pass  # Potentially a valid Q or A block with missing lines
            else:
                continue  # Irrelevant block; skip
        else:
            # Extract header and title lines
            header = lines[0].strip()  # e.g., "Erika Najarian Q"
            title = lines[1].strip()   # e.g., "Analyst, UBS Securities LLC"

        # Determine if the block is a Question or an Answer
        if header.endswith(' Q'):
            # If there's an existing question with answers, save it before proceeding
            if current_question is not None and current_answers:
                qna_list.append({
                    'Question': current_question,
                    'Answer': ' '.join(current_answers).strip(),
                    'Analyst': current_analyst,
                    'Respondent': ', '.join(sorted(current_respondents))
                })
                # Reset answers and respondents for the new Q&A pair
                current_answers = []
                current_respondents = set()
            
            # It's a Question block
            # Extract the Analyst's name by removing the trailing ' Q'
            analyst_name = header[:-2].strip()
            # Remove any numbers from the Analyst's name
            analyst_name = re.sub(r'\d+', '', analyst_name).strip()
            current_analyst = analyst_name
            
            # Extract the Question text (all lines after the first two lines)
            question_text = ' '.join(line.strip() for line in lines[2:])
            current_question = question_text
        
        elif header.endswith(' A'):
            # It's an Answer block
            # Extract the Respondent's name by removing the trailing ' A'
            respondent_name = header[:-2].strip()
            # Remove any numbers from the Respondent's name
            respondent_name = re.sub(r'\d+', '', respondent_name).strip()
            
            # Extract the Answer text (all lines after the first two lines)
            answer_text = ' '.join(line.strip() for line in lines[2:])
            current_answers.append(answer_text)
            current_respondents.add(respondent_name)
        
        else:
            # If the block doesn't start with ' Q' or ' A', ignore it
            continue
    
    # After processing all blocks, check if there's a pending Q&A to add
    if current_question is not None and current_answers:
        qna_list.append({
            'Question': current_question,
            'Answer': ' '.join(current_answers).strip(),
            'Analyst': current_analyst,
            'Respondent': ', '.join(sorted(current_respondents))
        })
    
    # Create the DataFrame from the list of Q&A dictionaries
    qna_df = pd.DataFrame(qna_list, columns=['Question', 'Answer', 'Analyst', 'Respondent'])
    
    # Optional: Clean excessive whitespace in 'Question' and 'Answer' columns
    qna_df['Question'] = qna_df['Question'].str.replace(r'\s+', ' ', regex=True).str.strip()
    qna_df['Answer'] = qna_df['Answer'].str.replace(r'\s+', ' ', regex=True).str.strip()
    
    return qna_df

def filter_qna_by_keywords(qna_df, keywords):
    """
    Filters the Q&A DataFrame to retain only rows where the Question or Answer contains any of the specified keywords.
    
    Parameters:
        qna_df (pd.DataFrame): The original Q&A DataFrame with columns ['Question', 'Answer', 'Analyst', 'Respondent'].
        keywords (list): List of keywords to filter by.
    
    Returns:
        pd.DataFrame: Filtered DataFrame containing only relevant Q&A pairs.
    """
    # Escape keywords to handle any special regex characters and join them with '|'
    escaped_keywords = [re.escape(word) for word in keywords]
    pattern = r'\b(?:' + '|'.join(escaped_keywords) + r')\b'
    
    # Compile the regex pattern for better performance
    regex = re.compile(pattern, re.IGNORECASE)
    
    # Create boolean masks for 'Question' and 'Answer' columns
    question_mask = qna_df['Question'].str.contains(regex, na=False)
    answer_mask = qna_df['Answer'].str.contains(regex, na=False)
    
    # Combine masks to identify rows where either 'Question' or 'Answer' matches
    combined_mask = question_mask | answer_mask
    
    # Apply the mask to filter the DataFrame
    filtered_qna_df = qna_df[combined_mask].reset_index(drop=True)
    
    return filtered_qna_df

Now we loop through all quarterly reports, generating a dataframe for combined management commentary/qna data

In [7]:
# Initialize an empty list to store DataFrames from each file
all_data = []

for file_path in file_paths:
    print(f"Processing file: {file_path}")
    
    # Step 1: Extract text and separate sections
    management_text, qna_text = separate_sections(file_path)
    
    # Step 2: Extract ROTCE sentences from management_text
    rotce_dicts = extract_rotce_sentences(management_text)
    rotce_sentences = [d['Answer'] for d in rotce_dicts if d['Answer']]
    
    # Step 3: Extract Q&A pairs and filter them for the relevant keywords
    qna_df = extract_qna_pairs_with_names(qna_text)
    filtered_qna_df = filter_qna_by_keywords(qna_df, keywords)
    
    # Step 4: Extract the quarter and year from the PDF filename (format: JPM_Qx_YYYY.pdf)
    quarter_match = re.search(r'JPM_(Q\d)_(\d{4})\.pdf', file_path, re.IGNORECASE)
    if quarter_match:
        # Reformat to 'YYYYQx' for proper sorting (e.g., '2022Q4')
        quarter = f"{quarter_match.group(2)}Q{quarter_match.group(1)[1:]}"
    else:
        quarter = 'Unknown'
    
    # Step 5: Create DataFrame for ROTCE sentences
    rotce_data = []
    for sentence in rotce_sentences:
        rotce_data.append({
            'Question': 'Management Commentary',
            'Answer': sentence,
            'Analyst': 'Management Commentary',
            'Respondent': 'Jeremy Barnum',
            'quarter': quarter
        })
    rotce_df = pd.DataFrame(rotce_data)
    
    # Step 6: Add 'quarter' column to filtered_qna_df
    filtered_qna_df['quarter'] = quarter
    
    # Ensure 'Respondent' column has no missing values
    filtered_qna_df['Respondent'] = filtered_qna_df['Respondent'].fillna('')
    
    # Reorder columns to match rotce_df and include 'quarter'
    filtered_qna_df = filtered_qna_df[['Question', 'Answer', 'Analyst', 'Respondent', 'quarter']]
    
    # Step 7: Combine ROTCE DataFrame with filtered Q&A DataFrame
    combined_df = pd.concat([filtered_qna_df, rotce_df], ignore_index=True)
    
    # Append the combined DataFrame to the master list
    all_data.append(combined_df)

# Step 8: Concatenate all DataFrames into a single DataFrame
master_df = pd.concat(all_data, ignore_index=True)

# Step 9: Convert 'quarter' column to a sortable Period type

master_df['quarter_period'] = pd.PeriodIndex(master_df['quarter'], freq='Q')

# Step 10: Sort the DataFrame by 'quarter_period' from oldest to newest
master_df = master_df.sort_values('quarter_period').reset_index(drop=True)

# Step 11: Drop the helper 'quarter_period' column
master_df = master_df.drop('quarter_period', axis=1)

Processing file: JPM_Q3_2024.pdf
Processing file: JPM_Q2_2024.pdf
Processing file: JPM_Q1_2024.pdf
Processing file: JPM_Q4_2023.pdf
Processing file: JPM_Q3_2023.pdf
Processing file: JPM_Q2_2023.pdf
Processing file: JPM_Q1_2023.pdf
Processing file: JPM_Q4_2022.pdf
Processing file: JPM_Q3_2022.pdf
Processing file: JPM_Q2_2022.pdf
Processing file: JPM_Q1_2022.pdf
Processing file: JPM_Q4_2021.pdf
Processing file: JPM_Q3_2021.pdf
Processing file: JPM_Q2_2021.pdf
Processing file: JPM_Q1_2021.pdf
Processing file: JPM_Q4_2020.pdf


In [9]:

# Define the ratio mapping with the format YYYYQx
ratios = {
    '2024Q3': 23405, '2024Q2': 22746, '2024Q1': 23082, '2023Q4': 24051, '2023Q3': 22726,
    '2023Q2': 21779, '2023Q1': 20711, '2022Q4': 20192, '2022Q3': 17518, '2022Q2': 15128,
    '2022Q1': 13872, '2021Q4': 13601, '2021Q3': 13080, '2021Q2': 12741, '2021Q1': 12889,
    '2020Q4': 13258
}

# Map the ratios to the 'quarter' column
master_df['ratio'] = master_df['quarter'].map(ratios)

# Export the final DataFrame to a CSV file
master_df.to_csv('combined_qna_nii_data.csv', index=False)

# Display the final combined DataFrame
master_df

,Question,Answer,Analyst,Respondent,quarter,ratio
0,Management Commentary,Declines in net interest income were largely o...,Management Commentary,Jeremy Barnum,2020Q4,13258
1,"So, I want to start off with a question on the...","Sure. So, I'll start with excess liquidity. So...",Steven Chubak,"Jamie Dimon, Jennifer A. Piepszak",2020Q4,13258
2,"Hi, Jenn. Given the outlook for net interest i...","Sure. So, I'll start by saying you're absolute...",John E. McDonald,Jennifer A. Piepszak,2020Q4,13258
3,"Okay. And then as a follow-up, on the NII walk...","Yes, it can absolutely be true. So, Markets is...",John E. McDonald,"Jamie Dimon, Jennifer A. Piepszak",2020Q4,13258
4,Management Commentary,So you can see that we do expect to be able to...,Management Commentary,Jeremy Barnum,2020Q4,13258
...,...,...,...,...,...,...
227,Management Commentary,5 billion and total NII to be approximately $92.,Management Commentary,Jeremy Barnum,2024Q3,23405
228,Management Commentary,"And given where we are in the year, we include...",Management Commentary,Jeremy Barnum,2024Q3,23405
229,Management Commentary,NII was $2.,Management Commentary,Jeremy Barnum,2024Q3,23405
230,Hey. Good morning. I'm not going to ask about ...,"Sure. Yeah, thanks for the question, Saul. And...",Saul Martinez,"Jamie Dimon, Jeremy Barnum",2024Q3,23405
